In [4]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN
from sklearn.experimental import enable_hist_gradient_boosting  # Required to use HistGradientBoosting
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Dropout, Reshape, Layer
from tensorflow.keras import backend as K

# Load Dataset
data = pd.read_csv(r"C:\Users\abdul\Desktop\research work\fetal_health.csv")  # Replace with the actual path

# Encode target variable
label_encoder = LabelEncoder()
data['fetal_health'] = label_encoder.fit_transform(data['fetal_health'])

# Split features and target
X = data.drop('fetal_health', axis=1)
y = data['fetal_health']

# Handle Imbalanced Dataset
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X, y)

adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_resampled, y_resampled)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, stratify=y_resampled, random_state=42)

# Scale Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(1,),
                                 initializer='zeros',
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)  # Calculate attention scores
        a = K.softmax(e, axis=1)              # Apply softmax to get attention weights
        output = x * a                        # Apply weights to input
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Neural Network with Attention (Using RMSprop Optimizer)
def build_nn_with_attention(input_dim, learning_rate=0.01, dropout_rate=0.3):
    model = Sequential([
        Dense(128, input_dim=input_dim),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        Dropout(dropout_rate),

        Reshape((1, 128)),  # Reshape to (batch_size, timesteps, features) for Attention
        Attention(),

        Dense(3, activation='softmax')  # Output Layer
    ])
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Measure Training Time for Neural Network with Attention
start_time_nn = time.time()
nn_model_with_attention = build_nn_with_attention(X_train_scaled.shape[1], learning_rate=0.01, dropout_rate=0.3)
nn_model_with_attention.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=150, batch_size=128, verbose=1)
end_time_nn = time.time()
nn_training_time = end_time_nn - start_time_nn
print(f"Training Time for Neural Network with Attention: {nn_training_time:.2f} seconds")

# Hyperparameter Tuning for HistGradientBoostingClassifier
hgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_iter': [100, 200, 300],
    'max_depth': [6, 8, 10],
    'min_samples_leaf': [20, 30, 40],
}
hgb = HistGradientBoostingClassifier(random_state=42)

# Measure Training Time for HistGradientBoosting
start_time_hgb = time.time()
grid_search_hgb = GridSearchCV(hgb, hgb_param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_hgb.fit(X_train_scaled, y_train)
end_time_hgb = time.time()
hgb_training_time = end_time_hgb - start_time_hgb
print(f"Training Time for HistGradientBoosting with Grid Search: {hgb_training_time:.2f} seconds")

# Stacking Ensemble
start_time_stack = time.time()
hgb_best = grid_search_hgb.best_estimator_
estimators = [('hgb', hgb_best)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)
stacking_model.fit(X_train_scaled, y_train)
end_time_stack = time.time()
stacking_training_time = end_time_stack - start_time_stack
print(f"Training Time for Stacking Model: {stacking_training_time:.2f} seconds")

# Measure Testing Time for Hybrid Model
start_time_test = time.time()
stacking_probs = stacking_model.predict_proba(X_test_scaled)
nn_probs_with_attention = nn_model_with_attention.predict(X_test_scaled)
final_probs_with_attention = (stacking_probs + nn_probs_with_attention) / 2
final_preds_with_attention = np.argmax(final_probs_with_attention, axis=1)
end_time_test = time.time()
testing_time = end_time_test - start_time_test
print(f"Testing Time for Hybrid Model with Attention: {testing_time:.2f} seconds")

# Evaluate Hybrid Model with Attention
accuracy_with_attention = accuracy_score(y_test, final_preds_with_attention)
print(f"Hybrid Model with Attention Accuracy: {accuracy_with_attention:.4f}")

print("\nClassification Report:")
target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test, final_preds_with_attention, target_names=target_names, digits=6))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, final_preds_with_attention))

# Compute evaluation metrics
accuracy = accuracy_score(y_test, final_preds_with_attention)
precision = precision_score(y_test, final_preds_with_attention, average="macro")
recall = recall_score(y_test, final_preds_with_attention, average="macro")
f1 = f1_score(y_test, final_preds_with_attention, average="macro")

print(f"\nHybrid Model Accuracy: {accuracy:.6f}")
print(f"Macro Precision: {precision:.6f}")
print(f"Macro Recall: {recall:.6f}")
print(f"Macro F1-score: {f1:.6f}")


Epoch 1/150
29/29 [==============================] - 2s 36ms/step - loss: 0.3855 - accuracy: 0.8382 - val_loss: 0.3107 - val_accuracy: 0.8785
Epoch 2/150
29/29 [==============================] - 0s 13ms/step - loss: 0.2733 - accuracy: 0.8848 - val_loss: 0.2664 - val_accuracy: 0.8901
Epoch 3/150
29/29 [==============================] - 0s 15ms/step - loss: 0.2270 - accuracy: 0.9080 - val_loss: 0.2358 - val_accuracy: 0.8983
Epoch 4/150
29/29 [==============================] - 0s 14ms/step - loss: 0.2093 - accuracy: 0.9165 - val_loss: 0.2142 - val_accuracy: 0.9017
Epoch 5/150
29/29 [==============================] - 0s 14ms/step - loss: 0.2089 - accuracy: 0.9154 - val_loss: 0.3249 - val_accuracy: 0.8570
Epoch 6/150
29/29 [==============================] - 0s 13ms/step - loss: 0.1769 - accuracy: 0.9286 - val_loss: 0.1877 - val_accuracy: 0.9289
Epoch 7/150
29/29 [==============================] - 0s 11ms/step - loss: 0.1724 - accuracy: 0.9344 - val_loss: 0.1652 - val_accuracy: 0.9314
Epoch 

NameError: name 'precision_score' is not defined